In [1]:
%load_ext autoreload

In [1]:
import pandas as pd
a = pd.read_feather("final_dataset.feather").set_axis(["user_id:token","item_id:token","rating:float","timestamp:float","status"],1)
a.head()

C:\Users\dtrag\AppData\Local\Temp\ipykernel_13376\72850041.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.set_axis except for the argument 'labels' will be keyword-only.
  a = pd.read_feather("final_dataset.feather").set_axis(["user_id:token","item_id:token","rating:float","timestamp:float","status"],1)


,user_id:token,item_id:token,rating:float,timestamp:float,status
0,Cezar10,48661,10,1648065035,2
1,Cezar10,43608,10,1656365421,2
2,Cezar10,48583,10,1649023161,2
3,Gonzo2,241,6,1293559897,2
4,Gonzo2,650,8,1293560393,2


In [11]:
b = a['item_id:token'].value_counts()
b[b>100].sum()

81253137

In [10]:
b = a['user_id:token'].value_counts()
b[b>4].sum()

81311025

In [13]:
mapper = dict(zip(a['user_id:token'].unique(),[*range(a['user_id:token'].nunique())]))


In [14]:
a['user_id:token'] = a['user_id:token'].map(mapper)

In [15]:
a.head()

,user_id:token,item_id:token,rating:float,timestamp:float,status
0,0,48661,10,1648065035,2
1,0,43608,10,1656365421,2
2,0,48583,10,1649023161,2
3,1,241,6,1293559897,2
4,1,650,8,1293560393,2


In [16]:
a = a.iloc[:,:4].set_index('user_id:token').astype("int32")

In [19]:
e = a.groupby("user_id:token",sort=False)["rating:float"].transform(pd.Series.mean)+10
a["rating:float"] = e

In [21]:
a.shape

(81443986, 3)

In [5]:
# b = pd.DataFrame(a.loc[:,"item_id:token"])

uniques = a['item_id:token'].unique()
mapper = dict(zip(uniques,range(len(uniques))))
# d = a['item_id:token'].map(mapper)
# b['train_id'] = d

pd.DataFrame(zip(uniques,[*range(len(uniques))]),columns=['item_id','train_id']).to_csv("item_map.csv",index=False)

# a['item_id:token'] = a['item_id:token'].map(mapper)
# a.head()
b.shape

(81443986, 2)

In [ ]:
a.to_csv("anime_atomic.inter",sep="\t")

In [15]:
sd = pd.read_csv("anime_atomic.inter",sep="\t")

In [18]:
sd[sd.iloc[:,0]==1]

,user_id:token,item_id:token,rating:float,timestamp:float
3,1,3,17.947368,1293559897
4,1,4,17.947368,1293560393
5,1,5,17.947368,1293560058
6,1,6,17.947368,1294080780
7,1,7,17.947368,1293559649
8,1,8,17.947368,1293560113
9,1,9,17.947368,1293560143
10,1,10,17.947368,1293559684
11,1,11,17.947368,1293560272
12,1,12,17.947368,1293560280
